# Creation of the dataset

This notebook contains the steps for the extraction and...

In [31]:
import os
import pymssql

import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

DATASET_CREATION = False

In [3]:
STATUS_PROTESTO = {
    1: "Incluída em lote de remessa",
    2: "Enviada a Protesto",
    3: "Protestada",
    4: "Paga",
    5: "Solicitação de Desistência",
    6: "Solicitação de Cancelamento (Após o Protesto)",
    7: "Solicitação de Autorização de Cancelamento (Dívida Paga ou Parcelada)",
    8: "Cancelada antes do Protesto",
    9: "Cancelada após o Protesto",
    10: "Cancelada por Pagamento",
    11: "Sustada por Ordem Judicial",
    12: "Devolvida por Irregularidade"
}

FINE_CODE_DICT = {
    1: "Ressarcimento",
    2: "Multa",
    3: "Remanejamento",
    4: "Multa Percentual",
    5: "Multa Cominatória"
}

STATUS_DIVIDA_ATIVA = {
    1: "Inscrito em Dívida Ativa",
    2: "Negociado",
    3: "Quitado",
    4: "Exigibilidade Suspensa",
    5: "Cancelado",
    6: "Pagamento em Atraso",
    7: "Remissão",
    8: "Prescrito"
}


In [4]:
def create_raw_dataset():
    queries = {
    "exe_debito": """
        SELECT 
            IdDebito AS exe_debito_iddebito,
            IdProcessoExecucao AS exe_debito_idprocessoexecucao,
            valorOriginalDebito AS exe_debito_valororiginaldebito,
            ValorPago AS exe_debito_valorpago,
            CodigoTipoDebito AS exe_debito_codigotipodebito,
            datainclusao AS exe_debito_datainclusao,
            DataDecisao,
            StatusProtesto AS protesto_status,
            Status_PGE AS pge_status,
            IdProcessoOrigem
        FROM processo.dbo.Exe_Debito
        
    """,
    "exe_debitopessoa": """
        SELECT 
            IDDebitoPessoa AS exe_debitopessoa_iddebitopessoa,
            IDDebito AS exe_debitopessoa_iddebito,
            IDPessoa AS exe_debitopessoa_idpessoa,
            DataInclusao AS exe_debitopessoa_datainclusao
        FROM processo.dbo.Exe_DebitoPessoa
    """,
    "exe_debitoboleto": """
        SELECT 
            IdDebitoBoleto AS exe_debitoboleto_iddebitoboleto,
            IdDebito as exe_debitoboleto_iddebito,
            ValorOriginal AS exe_debitoboleto_valororiginal,
            ValorPago AS exe_debitoboleto_valorpago,
            DataPagamento AS exe_debitoboleto_datapagamento
        FROM processo.dbo.Exe_DebitoBoleto
    """,
    "exe_debito_multadiaria": """
        SELECT 
            IdDebitoMultaDiaria AS exe_debito_multadiaria_iddebitomultadiaria,
            IdDebito as exe_debito_multadiaria_iddebito,
            ValorMultaDiaria AS exe_debito_multadiaria_valormultadiaria,
            DataInicioImputacaoMultaDiaria AS exe_debito_multadiaria_datainicio,
            DataFinalImputacaoMultaDiaria AS exe_debito_multadiaria_datafinal
        FROM processo.dbo.Exe_Debito_MultaDiaria
    """,
    "exe_creditopagamento": """
        SELECT 
            IdCreditoPagamento AS exe_creditopagamento_idcreditopagamento,
            IdDebito as exe_creditopagamento_iddebito,
            ValorCredito AS exe_creditopagamento_valorcredito,
            DataInclusao AS exe_creditopagamento_datainclusao
        FROM processo.dbo.Exe_CreditoPagamento
    """,
    "exe_parcelamento": """
        SELECT 
            IdParcelamento AS exe_parcelamento_idparcelamento,
            IdDebito as exe_parcelamento_iddebito,
            NumeroParcelas AS exe_parcelamento_numeroparcelas,
            SituacaoParcelamento AS exe_parcelamento_situacaoparcelamento,
            DataCancelamentoParcelamento AS exe_parcelamento_datacancelamento,
            DataReabertura AS exe_parcelamento_datareabertura
        FROM processo.dbo.Exe_Parcelamento
    """,
    "protesto_titulosremessa": """
        SELECT 
            IdTituloRemessa AS protesto_titulosremessa_idtituloremessa,
            IdDebito as protesto_titulosremessa_iddebito,
            NumeroProtocoloTitulo AS protesto_titulosremessa_numeroprotocolotitulo,
            DataProtocoloTitulo AS protesto_titulosremessa_dataprotocolotitulo
        FROM processo.dbo.Protesto_TitulosRemessa
    """,
    "pge_processo": """
        SELECT 
            IdProcessoPGE AS pge_processo_idprocessopge,
            IdDebitoExecucao as pge_processo_iddebitoexecucao,
            NumeroProcessoExecucao AS pge_processo_numeroprocessoexecucao,
            AnoProcessoExecucao AS pge_processo_anoprocessoexecucao,
            ValorAtualizadoPGE AS pge_processo_valoratualizadopge,
            ValorPagoPGE AS pge_processo_valorpagopge,
            HomologadoPGE AS pge_processo_homologadopge
        FROM processo.dbo.PGE_Processo
    """,
    "gen_pessoa": """
        SELECT 
            IdPessoa AS pessoa_idpessoa,
            CASE WHEN TipoPessoa = '2' THEN 1 ELSE 0 END AS is_legal_organization
        FROM processo.dbo.GenPessoa
    """,
    "processos": """
        SELECT 
            IdProcesso as processo_idprocesso,
            numero_processo,
            ano_processo
        FROM processo.dbo.Processos
    """,
    "processo_transitojulgado": """
        SELECT 
            numero_processo,
            ano_processo,
            datatransito AS processo_data_transito_julgado
        FROM processo.dbo.Processo_TransitoJulgado
    """,
    "civil_servant_match": """
        SELECT 
    CAST(ano AS INT) AS ano,
    CAST(mes AS INT) AS mes,
    gp.IdPessoa AS pessoa_idpessoa,
    1 AS is_civil_servant
    FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida fr 
    INNER JOIN processo.dbo.GenPessoa gp ON fr.CPF = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
    WHERE ano >= 2023
    GROUP BY ano, mes, gp.IdPessoa

        """
    }

    conn = pymssql.connect(
        server=os.getenv('SQL_SERVER_HOST'),
        port=int(os.getenv('SQL_SERVER_PORT')),
        user=os.getenv('SQL_SERVER_USER'),
        password=os.getenv('SQL_SERVER_PASS'),
        database=os.getenv('SQL_SERVER_DB'),
    )

    exe_debito = pd.read_sql(queries["exe_debito"], conn)
    exe_debitopessoa = pd.read_sql(queries["exe_debitopessoa"], conn)
    exe_debitoboleto = pd.read_sql(queries["exe_debitoboleto"], conn)
    exe_debito_multadiaria = pd.read_sql(queries["exe_debito_multadiaria"], conn)
    exe_creditopagamento = pd.read_sql(queries["exe_creditopagamento"], conn)
    exe_parcelamento = pd.read_sql(queries["exe_parcelamento"], conn)
    protesto_titulosremessa = pd.read_sql(queries["protesto_titulosremessa"], conn)
    pge_processo = pd.read_sql(queries["pge_processo"], conn)
    gen_pessoa = pd.read_sql(queries["gen_pessoa"], conn)
    processos = pd.read_sql(queries["processos"], conn)
    processo_transitojulgado = pd.read_sql(queries["processo_transitojulgado"], conn)
    civil_servant_match = pd.read_sql(queries["civil_servant_match"], conn)


    # Close the connection early
    conn.close()

    # Start with base Exe_Debito
    df = exe_debito.copy()

    # Join with Exe_DebitoPessoa
    df = df.merge(
        exe_debitopessoa,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_debitopessoa_iddebito'
    )

    # Join with Exe_DebitoBoleto
    df = df.merge(
        exe_debitoboleto,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_debitoboleto_iddebito'
    )

    # Join with Exe_Debito_MultaDiaria
    df = df.merge(
        exe_debito_multadiaria,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_debito_multadiaria_iddebito'
    )

    # Join with Exe_CreditoPagamento
    df = df.merge(
        exe_creditopagamento,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_creditopagamento_iddebito'
    )

    # Join with Exe_Parcelamento
    df = df.merge(
        exe_parcelamento,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_parcelamento_iddebito'
    )

    # Join with Protesto_TitulosRemessa
    df = df.merge(
        protesto_titulosremessa,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='protesto_titulosremessa_iddebito'
    )

    # Join with PGE_Processo
    df = df.merge(
        pge_processo,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='pge_processo_iddebitoexecucao'
    )

    # Join with GenPessoa
    df = df.merge(
        gen_pessoa,
        how='left',
        left_on='exe_debitopessoa_idpessoa',
        right_on='pessoa_idpessoa'
    )

    # Join with Processos
    df = df.merge(
        processos,
        how='left',
        left_on='IdProcessoOrigem',
        right_on='processo_idprocesso'
    )

    # Join with Processo_TransitoJulgado
    df = df.merge(
        processo_transitojulgado,
        how='left',
        on=['numero_processo', 'ano_processo']
    )

    # Extract year/month for civil servant matching
    df['ano'] = pd.to_datetime(df['DataDecisao'], errors='coerce').dt.year
    df['mes'] = pd.to_datetime(df['DataDecisao'], errors='coerce').dt.month

    # Join with civil servant match
    df = df.merge(
        civil_servant_match,
        how='left',
        on=['ano', 'mes', 'pessoa_idpessoa']
    )

    # Fill missing with 0
    df['is_civil_servant'] = df['is_civil_servant'].fillna(0).astype(int)

    df.to_csv('../data/private/raw/tcern_debtors_dataset.csv', index=False)

In [5]:
if DATASET_CREATION:
    create_raw_dataset()

In [6]:
id_columns_to_load_as_int = [
    'exe_debito_iddebito',
    'exe_debito_idprocessoexecucao',
    'exe_debitopessoa_iddebitopessoa',
    'exe_debitopessoa_idpessoa',
    'exe_debitoboleto_iddebitoboleto',
    'exe_debito_multadiaria_iddebitomultadiaria',
    'exe_creditopagamento_idcreditopagamento',
    'exe_parcelamento_idparcelamento',
    'protesto_titulosremessa_idtituloremessa',
    'pge_processo_idprocessopge',
    'protesto_titulosremessa_numeroprotocolotitulo'
]
dtype_map = {col: 'Int64' for col in id_columns_to_load_as_int}
df = pd.read_csv('../data/public/raw/tcern_debtors_dataset.csv', dtype=dtype_map)


/tmp/ipykernel_60939/1155430285.py:15: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/public/raw/tcern_debtors_dataset.csv', dtype=dtype_map)


In [43]:
df.columns


Index(['exe_debito_iddebito', 'exe_debito_idprocessoexecucao',
       'exe_debito_valororiginaldebito', 'exe_debito_valorpago',
       'exe_debito_codigotipodebito', 'exe_debito_datainclusao', 'DataDecisao',
       'protesto_status', 'pge_status', 'IdProcessoOrigem',
       'exe_debitopessoa_iddebitopessoa', 'exe_debitopessoa_iddebito',
       'exe_debitopessoa_idpessoa', 'exe_debitopessoa_datainclusao',
       'exe_debitoboleto_iddebitoboleto', 'exe_debitoboleto_iddebito',
       'exe_debitoboleto_valororiginal', 'exe_debitoboleto_valorpago',
       'exe_debitoboleto_datapagamento',
       'exe_debito_multadiaria_iddebitomultadiaria',
       'exe_debito_multadiaria_iddebito',
       'exe_debito_multadiaria_valormultadiaria',
       'exe_debito_multadiaria_datainicio', 'exe_debito_multadiaria_datafinal',
       'exe_creditopagamento_idcreditopagamento',
       'exe_creditopagamento_iddebito', 'exe_creditopagamento_valorcredito',
       'exe_creditopagamento_datainclusao', 'exe_parcelam

In [8]:
df.head()

,exe_debito_iddebito,exe_debito_idprocessoexecucao,exe_debito_valororiginaldebito,exe_debito_valorpago,exe_debito_codigotipodebito,exe_debito_datainclusao,DataDecisao,protesto_status,pge_status,IdProcessoOrigem,...,pge_processo_homologadopge,pessoa_idpessoa,is_legal_organization,processo_idprocesso,numero_processo,ano_processo,processo_data_transito_julgado,ano,mes,is_civil_servant
0,2,419587,13286.0,NaN,1,2014-11-04 08:18:28.330,2013-03-14,NaN,NaN,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
1,2,419587,13286.0,NaN,1,2014-11-04 08:18:28.330,2013-03-14,NaN,NaN,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
2,3,419587,2000.0,3393.88,2,2014-11-04 08:19:54.557,2013-03-14,NaN,True,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
3,3,419587,2000.0,3393.88,2,2014-11-04 08:19:54.557,2013-03-14,NaN,True,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
4,3,419587,2000.0,3393.88,2,2014-11-04 08:19:54.557,2013-03-14,NaN,True,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0


# Preprocessing

In [9]:
df['exe_parcelamento_situacaoparcelamento'].fillna(0, inplace=True)


/tmp/ipykernel_60939/3855748047.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['exe_parcelamento_situacaoparcelamento'].fillna(0, inplace=True)


In [10]:
df['exe_parcelamento_datacancelamento'].isna().sum()

np.int64(47026)

# Features for Clustering Debtor Profiles

## Financial Features
- `current_outstanding_balance`: Current net debt value (original debt minus paid amount).
- `percent_paid_amount`: Proportion of total original debt that has been paid.
- `num_distinct_debts`: Total count of unique debt obligations for the debtor.
- `total_multa_value`: Total original value of debts specifically identified as fines.
- `total_ressarcimento_value`: Total original value of debts identified as reimbursements.
- `num_boleto_payments`: Number of distinct payments made via boleto.
- `time_since_last_payment_days`: Time elapsed (in days) since the debtor's last recorded payment.

## Installment Features
- `has_ever_made_installment_agreement`: Binary flag (1 if yes, 0 if no) indicating if the debtor ever entered an installment agreement.

## Legal Process Features
- `has_ever_been_protested`: Binary flag (1 if yes, 0 if no) indicating if the debtor was ever subject to a notary protest for any of their debts.

## Debtor Profile Features
- `age_of_oldest_debt_days`: Time elapsed (in days) since the debtor's oldest recorded debt was included.

In [45]:
# Agrupamento principal por pessoa devedora
grouped = df.groupby('exe_debitopessoa_idpessoa')

# Calcula cada feature individualmente como Series
current_outstanding_balance = grouped.apply(
    lambda g: (g['exe_debito_valororiginaldebito'].fillna(0) - g['exe_debito_valorpago'].fillna(0)).sum()
)

percent_paid_amount = grouped.apply(
    lambda g: g['exe_debito_valorpago'].fillna(0).sum() / g['exe_debito_valororiginaldebito'].replace(0, 1).sum()
)

num_distinct_debts = grouped['exe_debito_iddebito'].nunique()

total_multa_value = grouped.apply(
    lambda g: g.loc[g['is_multa'] == 1, 'exe_debito_valororiginaldebito'].fillna(0).sum()
)

total_ressarcimento_value = grouped.apply(
    lambda g: g.loc[g['is_ressarcimento'] == 1, 'exe_debito_valororiginaldebito'].fillna(0).sum()
)

num_boleto_payments = grouped['exe_debitoboleto_iddebitoboleto'].nunique()

time_since_last_payment_days = (
    pd.Timestamp.now() - grouped['exe_debitoboleto_datapagamento'].max()
).dt.days

has_ever_made_installment_agreement = grouped['exe_parcelamento_idparcelamento'].apply(
    lambda x: int(x.notna().any())
)

has_ever_been_protested = grouped['protesto_titulosremessa_idtituloremessa'].apply(
    lambda x: int(x.notna().any())
)

age_of_oldest_debt_days = (
    pd.Timestamp.now() - grouped['exe_debito_datainclusao'].min()
).dt.days

# Combina todas as séries em um único DataFrame final
features_df = pd.concat([
    current_outstanding_balance.rename("current_outstanding_balance"),
    percent_paid_amount.rename("percent_paid_amount"),
    num_distinct_debts.rename("num_distinct_debts"),
    total_multa_value.rename("total_multa_value"),
    total_ressarcimento_value.rename("total_ressarcimento_value"),
    num_boleto_payments.rename("num_boleto_payments"),
    time_since_last_payment_days.rename("time_since_last_payment_days"),
    has_ever_made_installment_agreement.rename("has_ever_made_installment_agreement"),
    has_ever_been_protested.rename("has_ever_been_protested"),
    age_of_oldest_debt_days.rename("age_of_oldest_debt_days")
], axis=1)


/tmp/ipykernel_60939/3003757930.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  current_outstanding_balance = grouped.apply(
/tmp/ipykernel_60939/3003757930.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percent_paid_amount = grouped.apply(
/tmp/ipykernel_60939/3003757930.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future

In [50]:
features_df.reset_index(inplace=True)
features_df.drop(columns=['exe_debitopessoa_idpessoa'], inplace=True)

In [51]:
features_df.to_csv('../data/public/processed/tcern_debtors_features.csv', index=True)